# QoE Prediction Model Building


## 1. Compare Models in Federeted Learning Scenerio


Import libraries:


In [ ]:
from keras_core import Sequential
from keras_core.layers import LSTM, Dense, Dropout, Bidirectional, GRU, Input
from keras_core.metrics import MeanSquaredError
from typing import List, Tuple
from numpy.typing import NDArray
from functools import reduce
from numpy import add
import pandas as pd
import json
import matplotlib.pyplot as plt
import math

# weights
Layers = List[NDArray]
# weights and number of sample
ModelUpdate = Tuple[Layers, int]

Check if Keras can detect GPU:


In [ ]:
from jax.lib import xla_bridge

xla_bridge.get_backend().platform

Read dataset from file and split to samples and target:


In [ ]:
def read_data(file_id) -> Tuple[pd.DataFrame, pd.Series]:
    df = pd.read_csv(
        "./data/pokemon/chunk-{}.csv".format(file_id), index_col="id"
    ).drop(["user_id"], axis=1)
    x = df.drop(["MOS"], axis=1)
    y = df["MOS"]
    return x, y


def read_all_data():
    df = pd.read_csv("/model/data/pokemon.csv", index_col="id").drop(
        ["user_id"], axis=1
    )
    x = df.drop(["MOS"], axis=1)
    y = df["MOS"]
    return x, y

Prepare data for models initialization and evaluation:


In [ ]:
x_init, y_init = read_data("init")
x_test, y_test = read_data("test")

In [ ]:
x_init

Define RNN models:


In [ ]:
def lstm() -> Sequential:
    model = Sequential(
        layers=[
            Input((17, 1)),
            LSTM(128),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model


def bidirectional_lstm():
    model = Sequential(
        layers=[
            Input((17, 1)),
            Bidirectional(LSTM(128)),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model


def gru():
    model = Sequential(
        layers=[
            Input((17, 1)),
            GRU(128),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model

Utilities functions for train:


In [ ]:
def train(
    model: Sequential,
    weights: Layers,
    samples: pd.DataFrame,
    targets: pd.Series,
    epochs=20,
) -> ModelUpdate:
    model.set_weights(weights)

    model.fit(
        samples,
        targets,
        validation_split=0.05,
        batch_size=128,
        epochs=epochs,
        shuffle=True,
    )

    return model.get_weights(), len(samples)

In [ ]:
def fed_avg(updates: List[ModelUpdate]) -> Layers:
    num_examples_total = sum([num_examples for _, num_examples in updates])

    weights = [
        [layer * num_examples for layer in layers] for layers, num_examples in updates
    ]

    return [
        reduce(add, layer_updates) / num_examples_total
        for layer_updates in zip(*weights)
    ]

Init models with some data:


In [ ]:
fl_models = [lstm(), bidirectional_lstm(), gru()]
model_names = ["LSTM", "Bidirectional LSTM", "GRU"]
init_weights = [
    train(model, model.get_weights(), x_init, y_init)[0] for model in fl_models
]
fl_result = {}
for name in model_names:
    fl_result[name] = []

In each round, each model will be trained with 25 dataset separately, then 25 sets of weights will be aggregated


In [ ]:
nor = 5  # number of round
dpr = math.floor(150 / nor)  # datasets per round

for round in range(nor):
    datasets = [read_data(dpr * round + i) for i in range(1, 1 + dpr)]

    for name, model in zip(model_names, fl_models):
        old_weights = model.get_weights()
        local_updates = [train(model, old_weights, x, y) for (x, y) in datasets]
        new_global_weights = fed_avg(local_updates)
        model.set_weights(new_global_weights)
        result = model.evaluate(x_test, y_test)[0]
        fl_result[name].append(result)

In [ ]:
json_result = json.dumps(fl_result, indent=4)
print(json_result)

In [ ]:
plt.plot(fl_result["LSTM"], "o-r", label="lstm")
plt.plot(fl_result["Bidirectional LSTM"], "o-b", label="bidirectional lstm")
plt.plot(fl_result["GRU"], "o-g", label="gru")

x = [dpr * i for i in range(1, 1 + nor)]
xi = list(range(len(x)))
plt.xticks(xi, x)

plt.xlabel("Number of Client")
plt.ylabel("MSE")

plt.legend()
plt.show()

In [ ]:
cl_model = lstm()
cl_model.set_weights(init_weights[0])

In [ ]:
cl_result = []

In [ ]:
for round in range(nor):
    datasets = [read_data(i) for i in range(1, dpr * (round + 1))]
    x = pd.concat([x for x, _ in datasets], axis=0)
    y = pd.concat([y for _, y in datasets], axis=0)
    train(cl_model, init_weights[0], x, y)
    result = cl_model.evaluate(x_test, y_test)[0]
    cl_result.append(result)

In [ ]:
plt.plot(cl_result, "o-r", label="Central")
plt.plot(fl_result["LSTM"], "o-b", label="FedAvg")
x = [dpr * i for i in range(1, 1 + nor)]
xi = list(range(len(x)))
plt.xticks(xi, x)

plt.xlabel("Number of Client")
plt.ylabel("MSE")

plt.legend()
plt.show()